In [82]:
import mesa
from functional import seq
import random

from dataclasses import dataclass

In [83]:
from typing import Tuple, Optional, List

In [84]:
@dataclass
class Hotel():
    id: int
    cost: float
    max_capacity: int
    occupied: int = 0

    # stats
    new_rev: int = 0
    total_rev: int = 0

    def remaining_capacity(self) -> int:
        return self.max_capacity - self.occupied

In [85]:
@dataclass
class PointOfInterest():
    id: int
    cost: float
    max_capacity: int
    occupied: int = 0

    # stats
    visit: int = 0
    total_visit: int = 0

    def remaining_capacity(self) -> int:
        return self.max_capacity - self.occupied

In [79]:
class Tourist(mesa.Agent):
    def __init__(self, unique_id: int, model, loc: int, trip_length: int):
        super().__init__(unique_id, model)
        self.loc: int = loc
        self.hotel: Optional[Hotel] = None
        self.poi:Optional[PointOfInterest]=None
        self.trip_length: int = trip_length
        self.remaining_life: int = trip_length

        self.satisfaction: int = 0

    def step(self):
        self.remaining_life -=1
        if self.hotel is None and self.remaining_life >= 3:
            print("Looking for hotel")
            hotels = (
                seq(self.model.hotels)
                    .filter(lambda h: h.remaining_capacity() >= 0)
                    .list()
            )

            if len(hotels) <= 0:
                self.remaining_life = -1
                return;

            random.shuffle(hotels)

            self.hotel = hotels[0]
            self.hotel.occupied += 1
            self.hotel.new_rev += 1
            self.hotel.total_rev += 1

            return

        if self.poi is None:
            print("Looking for point of interest")
            no_capacity_poi=[p for p in self.model.poi if p.remaining_capacity()==0]


            if len(no_capacity_poi) > 0:
                print("NO Point of Interest")
                self.satisfaction -= 1
                print(f"self satisfaction: {self.satisfaction}")
                return self.satisfaction
            
            random.shuffle(self.model.poi)
            self.poi = self.model.poi[0]
            
            return
        
            


In [77]:
class Model(mesa.Model):
    def __init__(self, num_tourists, num_hotels, num_of_poi):
        self.hotels: List[Hotel] = []
        self.poi: List[PointOfInterest] = []
        self.tourists: List[Tourist] = []

        self.schedule=mesa.time.RandomActivation(self)

        # Stats
        
        
        # should be replaced with data from file
        for i in range (num_of_poi):
            self.hotels.append(Hotel(i, 200, 100))

        for i in range (num_hotels):
            self.poi.append(PointOfInterest(i, 200, 100))
            # self.schedule.add(hotel)

        for i in range(num_tourists):
            tourist = Tourist(i, self, 0, 10)
            self.tourists.append(tourist)
            self.schedule.add(tourist)

    def step(self):
        # self.schedule.step()
        # self.schedule.step()
        # self.schedule.step()

        self.update_tourists()
        self.update_tourists()
        self.update_tourists()

        self.remove_tourists()

        print("stats")
        print(self.hotel_stats())

    def update_tourists(self):
        random.shuffle(self.tourists)

        for tourist in self.tourists:
            tourist.step()



    def remove_tourists(self):
        # (
        #     seq(self.model.schedule.agents)
        #         .filter()
        # )
        remove_index: list[int] = (seq(self.tourists)
            .enumerate()
            .filter(lambda x: x[1].remaining_life < 0)
            .map(lambda x: x[0])
            .sorted(reverse=True)
            .list()
        )
        for i1 in remove_index:
            self.tourists[i1].hotel.occupied -=1

            del self.tourists[i1]

    def hotel_stats(self) -> Tuple[float, float, float]:
        total_capacity: int = (seq(self.hotels)
            .map(lambda h: h.max_capacity)
            .fold_left(0, lambda acc, next: acc + next)
        )
        empty_rooms: int = (seq(self.hotels)
            .map(lambda h: h.remaining_capacity())
            .fold_left(0, lambda acc, next: acc + next)
        )
        occupied_rooms: int = (seq(self.hotels)
            .map(lambda h: h.occupied)
            .fold_left(0, lambda acc, next: acc + next)
        )
        
        new_reservations: int = (seq(self.hotels)
            .map(lambda h: h.new_rev)
            .fold_left(0, lambda acc, next: acc + next)
        )
        for hotel in self.hotels:
            hotel.new_rev = 0

        return (total_capacity, empty_rooms, occupied_rooms, new_reservations)
    
    def poi_stats(self) -> Tuple[float]:
        total_capacity_poi: int = (seq(self.poi)
            .map(lambda h: h.max_capacity)
            .fold_left(0, lambda acc, next: acc + next)
        )
        return total_capacity_poi

In [80]:
starter_model=Model(3, 3, 3)
for i in range(20):
    starter_model.step()

Looking for hotel
Looking for hotel
Looking for hotel
Looking for point of interest
NO Point of Interest
self satisfaction: -1
Looking for point of interest
NO Point of Interest
self satisfaction: -1
Looking for point of interest
NO Point of Interest
self satisfaction: -1
Looking for point of interest
NO Point of Interest
self satisfaction: -2
Looking for point of interest
NO Point of Interest
self satisfaction: -2
Looking for point of interest
NO Point of Interest
self satisfaction: -2
stats
(300, 0, 3, 3)
Looking for point of interest
NO Point of Interest
self satisfaction: -3
Looking for point of interest
NO Point of Interest
self satisfaction: -3
Looking for point of interest
NO Point of Interest
self satisfaction: -3
Looking for point of interest
NO Point of Interest
self satisfaction: -4
Looking for point of interest
NO Point of Interest
self satisfaction: -4
Looking for point of interest
NO Point of Interest
self satisfaction: -4
Looking for point of interest
NO Point of Interes

C:\Users\arine\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)
